In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep
import pandas as pd

# Webtoon data crawling

- collect : title, author, day, genre, story(text description), platform, age

In [ ]:
## 1. HTML source of webtoon page


URL = 'https://comic.naver.com/webtoon/weekday'
html = requests.get(URL).text
soup = BeautifulSoup(html, 'html.parser')

## 2. Collect the title and list generation

title = soup.find_all('a', {'class' : 'title'}) 
id_list = [] ; title_list = [] ; author_list = [] ; day_list = [] 
genre_list = [] ; story_list = [] ; platform_list = []; age_list = []
num = 0

## 3. Open the page using webdriver

driver = webdriver.Chrome('location of the chrome driver') 
driver.get(URL)

## 4. Crawling


for i in range(len(title)):
    sleep(0.5) 
    
    page = driver.find_elements_by_class_name('title')
    page[i].click()
    sleep(0.5)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser') 

    day = soup.find_all('ul', {'class' : 'category_tab'})
    day = day[0].find('li', {'class' : 'on'}).text[0:1] 

    t = title[i].text
    if (t in title_list):  
        day_list[title_list.index(t)] += ', ' + day
        driver.back()
        continue
        
    id_list.append(i) ; num += 1  
    title_list.append(t)  
    day_list.append(day) 
    platform_list.append('Naver webtoon') 

    author = soup.find_all('h2') 
    author = author[1].find('span', {'class' : 'wrt_nm'}).text[8:] 
    author_list.append(author) 

    genre = soup.find('span', {'class' : 'genre'}).text 
    genre_list.append(genre) 

    story = soup.find_all('p') 
    story = str(story[3])
    story = story.replace('<p>', '').replace('</p>', '').replace('<br/>', '\n') 
    story_list.append(story) 
    
    age = soup.find('span',{'class':'age'})
    age_list.append(age)
    

    driver.back() 
    

## 5. Dataframe Generation

cols = []
webtoon = pd.DataFrame(columns = cols)
webtoon['id'] = id_list
webtoon['title'] = title_list
webtoon['author'] = author_list
webtoon['day'] = day_list
webtoon['genre'] = genre_list
webtoon['story'] = story_list
#webtoon['platform'] = platform_list
webtoon['age'] = age_list
webtoon['age'] = webtoon['age'].astype(str)

for i in range(len(webtoon)):
    webtoon.loc[i,'age'] = (webtoon.loc[i,'age']).replace('<span class="age">', '').replace('</span>', '')

webtoon.to_csv('naver_webtoon.csv', encoding = 'utf-8-sig')




## Finished webtoon

In [ ]:
## 1. HTML source of webtoon page

URL = 'https://comic.naver.com/webtoon/finish?view=list&order=User'
html = requests.get(URL).text 
soup = BeautifulSoup(html, 'html.parser')

## 2. List generation

#title = soup.find_all('a') 
id_list = [] ; title_list = [] ; author_list = [] ; day_list = [] 
genre_list = [] ; story_list = [] ; platform_list = []; age_list = []
num = 0

## 3. Open the page using webdriver

driver = webdriver.Chrome('location of the chrome driver') 
driver.get(URL)

## 4. Crawling 

for i in range(1,1159):
    sleep(0.5) 
    
    webtoon = '//*[@id="content"]/div[2]/table/tbody/tr[{}]/td[1]/a/strong'.format(i)
    button = driver.find_element_by_xpath(webtoon)
    button.click()
    sleep(0.8)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser') 

    id_list.append(i)
    
    title = soup.find('span','title')
    title_list.append(title.get_text())  

    platform_list.append('네이버 웹툰') 

    author = soup.find_all('h2') 
    author = author[1].find('span', {'class' : 'wrt_nm'}).text[8:] 
    author_list.append(author) 
    
    sleep(0.2)

    genre = soup.find('span', {'class' : 'genre'}).text 
    genre_list.append(genre) 

    story = soup.find_all('p') 
    story = str(story[3])
    story = story.replace('<p>', '').replace('</p>', '').replace('<br/>', '\n') 
    story_list.append(story) 
    
    age = soup.find('span',{'class':'age'})
    
    if age == None:
        age_list.append(' ')
    else:    
        age_list.append(age.get_text())
    

    driver.back() 
    

## 5. DataFrame Generation

cols = []
webtoon2 = pd.DataFrame(columns = cols)
webtoon2['id'] = id_list_final
webtoon2['title'] = title_list_final
webtoon2['author'] = author_list_final
#webtoon2['day'] = day_list_final
webtoon2['genre'] = genre_list_final
webtoon2['story'] = story_list_final
webtoon2['platform'] = platform_list_final
webtoon2['age'] = age_list_final

webtoon2.to_csv('naver_webtoon2.csv', encoding = 'utf-8-sig')

# Data concatenate

- we concatnate the webtoon1 and webtoon2

In [ ]:
## 1. Data concatenation

web_df = pd.concat([naver_1,naver_2])
web_df.info()

## 2. Drop the unused column and null

# day 
web_df.drop('day', axis = 1,inplace = True)

 
web_df.replace(' ', np.nan, inplace = True)
web_df.dropna(axis = 0,inplace = True)

# age label preprocessing
web_df['age'] = web_df['age'].str.replace("전체 이용가","전체연령가")

web_df.reset_index(drop = True, inplace = True)


# 'age_all' column
web_df = pd.get_dummies(data = web_df,columns = ['age'])
web_df.rename(columns={'age_전체연령가' : 'age_all'},inplace = True)
web_df['age_all'] = web_df['age_all'].astype('int')

# Thumbnail collection

In [ ]:
## 1. Obtain title list

title_list = naver['title'].to_list()

## 2. HTML source of webtoon page

URL = 'https://comic.naver.com/index'
driver = webdriver.Chrome()
driver.get(URL)
html = driver.page_source
soup = BeautifulSoup(html,'html.parser')

## 3. Crawling

for title in title_list:
    
    search_box = driver.find_element_by_id('gnb.keyword')
    search_box.clear()

    search_box.send_keys(title)
    
    search_click = driver.find_element_by_id('search_bar_button')
    search_click.click()

    sleep(2)

    webtoon = driver.find_element(By.XPATH,'//*[@id="content"]/div[2]/ul/li/h5/a')
    webtoon.click()

    sleep(2)

    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')

    img = soup.find('div',{'class' : 'comicinfo'}).find('img')
    
    if '/' in title :
        title = title.replace('/','_')
        
    savename = 'file save location' + title + '.jpg'
    img_src = img['src']
    urlretrieve(img_src, savename)
    
    driver.back()
    driver.back()
    sleep(1)
    
